In [ ]:
!pwd
import sys
import os
path = os.getcwd()
path += '/../../'
sys.path.append(path)

import starburst
from starburst import utils
import log_jobs
import submit_jobs
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import time 
import copy 
from collections import OrderedDict 
import math
import heapq
import re
from IPython.display import display
import itertools
import pickle
pd.set_option('display.max_columns', None)

In [ ]:
VMS = {
    "05_01_2023": 1682925843,
    "05_02_2023": 1683099273,
    "05_03_2023": 1683132152,
    "05_03_2023_2": 1683173965,
    "05_04_2023_1": 1683184792, 
    "05_04_2023_2": 1683196845,
    "05_04_2023_3": 1683259533,
    "05_05_2023_4": 1683343491,
    "05_06_2023_4": 1683410859,
    "05_06_2023_5": 1683418719,
    "05_06_2023_6": 1683442154,
    "05_06_2023_7": 1683452077,
    "05_07_2023_1": 1683486242,
    "05_07_2023_2": 1683489564,
    "05_07_2023_3": 1683496107,
    "05_07_2023_4": 1683497538,
    "1683498468": 1683497538, 
    "1683498857": 1683498857,
    "1683499283": 1683499283,
    "1683528723": 1683528723, # Fixed JCT values
    "1683534589": 1683534589, # Large sweep with greater arrival rates
    "1683607638": 1683607638, # Modified arrival rate values to match simulator
    "1683625643": 1683625643, # Policy waits until cluster state is updated
    "1683627780": 1683627780, # Policy waits until cluster state and running pods are updated
    "1683680278": 1683680278, # Sweep with uniform wait (4 sec) and constant timeout (3 sec)
    "1683705558": 1683705558, # 30 second inter arrival time and 5 second job time
    "1683707012": 1683707012, # 30 second inter arrival time and 5 second job time (ROUND 2)
    "1683757260": 1683757260, # 30 second inter arrival time and 5 second job time (ROUND 3)
    "1683758384": 1683758384, # 5 second inter arrival time and 5 second job time (ROUND 3)
    "1683759834": 1683759834, # 5 second arrival 5 second job 1 second wait
    "1683939895": 1683939895, 
    "1683959890": 1683959890, # Waiting until all jobs submitted
    "1683961793": 1683961793, # Waiting until jobs submitted and relogging with poroper config second time
    "1683999940": 1683999940, # Fixed unscheduled gpu pod error
    "1684025542": 1684025542, # Resolved last job logging error
    "1684027483": 1684027483, # Decreased interarrival times
    "1684034550": 1684034550, # Replace gpu index with uuid
    "1684035873": 1684035873, # Varying interarrival rate of gpu jobs
    "1684038713": 1684038713, # Included GPUS per node value to sweep and fixed values [PLOTS WORK]
    "1684107011": 1684107011, # Run with varying timeout values to test cost saving plots
    "1684127501": 1684127501, # Higher range of arrival rates (from 3 arrival rates to 5 rates) 
    "1684211347": 1684211347, # Use loop_time instead of curr_time when comparing job wait_time
    "1684211782": 1684211782, # Doesn't check if job on queue is less than timeout, when submitting on prem
    "1684214027": 1684214027, # Dropped schedule tick from 1 to 0.1
    "1684219829": 1684219829, # Fixed checking cluster state if job completed -- set to 0.5 sched tick
    "1684222781": 1684222781, # Interarrival rate to 3 seconds 
    "1684223508": 1684223508, # Rerun 1684107011 log w/ 3 inter arrival and 10 wait
    "1684279856": 1684279856, # Checks if previous job submitted has been scheduled to avoid onprem congestion
    "1684282049": 1684282049, # Removes the 50 length limit from http_info +  Handles case when next job not found in the cluster yet -- moves onto next job in the meantime
    "1684282536": 1684282536, # Removed 50 limit for both check cluster state and get if last job scheduled
    "1684283565": 1684283565, # Handling tuple jobname and jobstatus error
    "1684295134": 1684295134, # Read the logs
    "1684300887": 1684300887, # Added loop time and dropped schedule tick event
    "1684301139": 1684301139, # Removed Job Add Event
    "1684301369": 1684301369, # Removed everything except process_queue
    "1684301655": 1684301655, # No 0.5 pause within the event loop
    "1684307219": 1684307219, # New VM
    "1684311915": 1684311915, # Debugged new VM
    "1684312326": 1684312326, # Loop time
    "1684312971": 1684312971, # Process Queue and Event Queue Times
    "1684313238": 1684313238, # Log time
    "1684313922": 1684313922, # Queue Add Time
    "1684366876": 1684366876, # Fixed interloop time to 1 second 
    "1684368494": 1684368494, # Fixed loop time between loops
    "1684369123": 1684369123, # Rerun because jobs corrupted
    "1684371614": 1684371614, # Tried to ensure all previous running jobs are deleted
    "1684380753": 1684380753, # Sweep different wait time values
    "1684382045": 1684382045, # Multi Node 8 GPU 
    "1684397749": 1684397749, # Removed regex for scheudled_node event
    "1684433340": 1684433340, # Modified onprem submission conditional
    "1684445674": 1684445674, # Event tracked in events.log
    "1684449505": 1684449505, # Testing new gpu_index parsing code -- waiting till run finishes 
    "1684452105": 1684452105, # 60 jobs modified policy all onprem
    "1684452497": 1684452497, # 360 jobs on prem
    "1684470801": 1684470801, # New terminating conditions
    "1684471163": 1684471163, # Added end variable loop
    "1684471393": 1684471393, # Cloud spill new termination
    "1684471749": 1684471749, # Large spillover 
    "1684473449": 1684473449, # Check cloud spill over policy only 
    "1684475422": 1684475422, # Cloud only with 4 gpus
    "1684477231": 1684477231, # Cloud only with 4 gpus without setup script bug
    "1684478946": 1684478946, # Spill over from cloud to onprem 4 nodes each -- constant wait 10 seconds
    "1684480072": 1684480072, # Constant wait 30 seconds -- 4 onprem, 4 cloud, 60 second batch
    "1684480621": 1684480621, # Constant wait 60 seconds -- 4 onprem, 4 cloud, 60 second batch 
    "1684486650": 1684486650, # Constant wait 5 seconds -- 4 onprem, 4 cloud, 60 second batch
    "1684487285": 1684487285, # Wait 0, 1, 2 seocnds -- 4 onprem, 4 cloud, 60 second batch
    "1684489189": 1684489189, # Wait 2, 3, 5, 10 seconds -- 30 second mean duration
    "1684532430": 1684532430, # Sweep over different poicies 
    "1684534781": 1684534781, # Philly trace sweep 
    "1684536405": 1684536405, # Reduced uniform arrival 
    "1684537269": 1684537269, # 30 second average job duration
    "1684538298": 1684538298, # Modified compute wait threshold
    "1684543740": 1684543740, # Integrating chakra 
    "1684545298": 1684545298, # New node name
    "1684548233": 1684548233, # Updated chakra and sweep of final jobs (starburst', 'compute_optimal', 'constant_optimal', 'constant')
    "1684609979": 1684609979, # Working chakra, mnist with starburst
    "1684611837": 1684611837, # Pending 4 gpu job error
    "1684621271": 1684621271, # Removed cloud cluster
    "1684630465": 1684630465, # Removed cloud cluster w/ modified logs
    "1684720040": 2,
    "1684726518": 2,
    "1684704725": 1, # Real sweep of training jobs from Ryan
    "1684726518": 2, 
    "1684727264": 2, 
    "1684731034": 2, # More jobs ~ 15 minutes each
    "1684755479": 2, # overloaded sys util - 20 jobs/sec
    "1684754864": 1, # high sys util - arrival 12 jobs/sec
    "1684795422": 2, # overloaded sys util - 32 jobs/sec
    "1684795437": 1, # high sys util - arrival 24 jobs/sec
    "1684804876": 2, # overloaded with continous arrival time logs
    "1684804861": 1, # high sys util with continous arrival time logs,
    "1684807867": 2, # overloaded 
    "1684808264": 1, # high load
    "1684814611": 2, # overloaded
    "1684814629": 1, # high sys
    "1684825502": 1, # high sys
    "1684842795": 2, # Final experiment high sys util
    "1684842093": 1, # Final experiment overloaded sys util # Poster data
    "1686798842": 1, # CPU Sleep jobs 
    "1687837800": 1, # CPU Sleep jobs w/ simulated cloud logging
}
# TODO: Support both GPU and CPU (e.g. "workload_type")
logs =  "1687837800"#"1686798842"#"1684842093"#"1684842795"#"1686798842"# "1684842093"#"1686798842" #"1684842795" ##"1684842795" #"1684755479" #"1684754864" #"1684731034" #LOGS["1684630465"]

In [ ]:
# TODO: Check for ground truth  -- Compare the datapoints from parse_job_df to ensure there are no missing jobs 

def parse_job_df(cluster_event_df=None, submission_df=None, sweep_df=None, avoid_congestion=True, columns=None, run_id=None, cloud_log_list=None):
    '''
    Parse onprem jobs first
    '''
    hyperparameters = None
    if 'hyperparameters' in submission_df: 
        hyperparameters = submission_df['hyperparameters']
    
    onprem_event_df = cluster_event_df['onprem']
    cloud_event_df = cluster_event_df['cloud']
    
    job_names = {}
    jobs = {}
    for col in columns: 
        jobs[col] = []

    all_nodes = set()
    nodes = {}
    node_counter = 0
    types = ['onprem', 'cloud']
    nodes_indices = {}
    
    #import pdb; pdb.set_trace()
    print(cloud_log_list)
    
    for cluster_type in types: 
        event_df = cluster_event_df[cluster_type]
        try:
            job_times = {}
            pod_times = {}
            if cluster_type == 'cloud':
                for job in cloud_log_list[1:-1]:
                    match = re.search(r"Cloud Job \|\| job id (\S+) \| job name (\S+) \| estimated cloud start time (\S+) \| estimated job duration (\S+) \| submission time (\S+) \| gpus (\S+) \| cpus (\S+)", job)
                    match = match.groups()
                    if match:
                        job_id = int(match[0])
                        job_name = str(match[1])
                        submit = float(match[2])
                        duration = float(match[3])
                        arrival = float(match[4])
                        gpu = int(match[5])
                        cpu = int(match[6])
                        end = submit + 2 + duration 
                        
                        times = {"arrival": arrival,"submit": submit,"pod_start": submit + 2, "job_end": end, "gpu": gpu, "cpu":cpu}
                        print(f'matched time {times}')

                        missing = False
                        for v, k in times.items(): 
                            if not k: 
                                print(f'Missing {v} {k}')
                                missing = True
                        print(f'value {times}')
                    
                        pod_times[job_id] = times
                intervals = pod_times
                nodes["cloud"] = node_counter
                #print(pod_times)
                print(f'Cloud Jobs {len(intervals)}')       
            else:   
                cluster = event_df
                start_times = cluster['container_start_times']
                creation_times = cluster['job_creation_times']
                completion_times = cluster['job_completion_times']
                # TODO: Verify node names works as intended
                pod_nodes = cluster['node_name']
                job_pods = cluster['job_pods']
                pod_jobs = {value: key for key, value in job_pods.items()}
                node_instances = cluster['node_instances']

                job_start_times = {}
                job_end_times = {}
                pod_start_times = {}

                for pod in start_times:
                    pod_name = pod
                    pod_start_time = start_times[pod]
                    pod_start_times[pod_name] = pod_start_time   

                for job in creation_times:
                    job_name = job
                    job_start_time = creation_times[job]
                    job_start_times[job_name] = job_start_time

                for job in completion_times:
                    job_name = job
                    job_end_time = completion_times[job]
                    job_end_times[job_name] = job_end_time
                       
                for pod in pod_nodes: 
                    all_nodes.add(pod_nodes[pod])

                for job in job_start_times:
                    if job in job_end_times:
                        job_times[job] = [job_start_times[job], job_end_times[job]]
                    
                    job_name = job
                    if job_name in {'job-87', 'job-85'}: 
                            if job_name in job_start_times and job_name not in job_end_times: 
                                #1684842795
                                missing_times = {
                                    'job-87': 7897,
                                    'job-85': 8999

                                }
                                job_end_times[job_name] = job_start_times[job_name] + missing_times[job_name]

                for pod in pod_start_times:
                    if pod in pod_jobs:
                        job = pod_jobs[pod]
                        if job not in job_end_times:
                            print("MISSING JOB " + str(job))
                        job_id = re.findall(r'\d+', job)[0]
                            
                        sub_time = submission_df[job_id]['scheduler_submit_time']
                        if job not in job_end_times: 
                            job_end_times[job] = sub_time + 100000
                            print("MISSING JOB " + str(job))
                            
                        pod_times[job] = [sub_time, job_end_times[job]]
                        #import pdb; pdb.set_trace()

                        times = {"arrival": submission_df[job_id]['scheduler_submit_time'], \
                                 # Submit is also job start time
                                 "submit": job_start_times[job], \
                                 "pod_start": pod_start_times[pod], \
                                 "job_end": job_end_times[job], \
                                }
                        print(f'matched time {times}')
                        missing = False
                        for v, k in times.items(): 
                            if not k: 
                                print(f'Missing {v} {k}')
                                missing = True
                                #continue
                        #if not missing:
                        print(f'value {times}')
                        if not times['job_end']:
                            times['job_end'] = times['submit'] + 1000000
                            
                        if job_id in {'87', '85'}: 
                            missing_runtime = times['job_end'] - times['pod_start']
                            print(f'MISSING RUNTIME {job_id} {missing_runtime}')
                        pod_times[job] = times
                        #else: 
                            #print("MISSING JOB " + str(job))

                # TODO: Possible cause of error 
                for n in all_nodes:
                    nodes[n] = node_counter
                    node_counter += 1

                intervals = pod_times
                print(f'Onprem Jobs {len(intervals)}')
                
        except Exception as e:
            import traceback
            print(f'matched time {times}')
            trace = traceback.format_exc()
            print(trace)

        gpu_indices = {}
        gpu_num = 0
        gpu_nums = {}
        gpu_node = {} #mapping from gpu_uuid to node_name
        node_gpus = {} #mapping from node_name to gpu_uuid to int [0, 7]
        print(f'interval {intervals}')
        for i, (key, value) in enumerate(intervals.items()):
            try:
                print(f'{cluster_type} value {value}')
                if cluster_type == 'cloud':
                    #print(key)
                    job_id = str(key)
                else:
                    job_id = re.findall(r'\d+', key)[0] #e.g. "sleep-26-100444"
                job_names[i] = key
                jobs['idx'].append(int(job_id))
                jobs['runtime'].append(value['job_end']-value['pod_start'])
                jobs['start'].append(value['pod_start'])
                jobs['arrival'].append(value['arrival'])
                jobs['submission_time'].append(value['submit'])
                #print(f'{cluster_type} value {value}')
                
                if cluster_type == 'cloud':
                    #print(value)
                    jobs['num_gpus'].append(value['gpu'])
                    #TODO: Verify cpu value for cloud jobs
                    jobs['cpus'].append(value['cpu'])
                else:
                    jobs['num_gpus'].append(submission_df[job_id]['workload']['gpu'])
                    jobs['cpus'].append(submission_df[job_id]['workload']['cpu'])

                # TODO: Cleanup
                if avoid_congestion:
                    submit_time = cluster['job_creation_times'][key] #Job start time
                else:
                    submit_time = submission_df[job_id]['scheduler_submit_time'] #Job submission time
                
                if not submit_time:
                    jobs['wait_times'].append(0)
                else:
                    jobs['wait_times'].append(value['pod_start']-value['arrival'])

                if cluster_type == "cloud":
                    jobs['is_local'].append(0)
                else:
                    jobs['is_local'].append(1)
                
                #if cluster_type == 'cloud':
                
                if cluster_type == 'cloud':
                    jobs['instance_type'].append("unknown")
                    jobs['node'].append("cloud")
                    jobs['allocated_gpus'].append({})
                    jobs['node_index'].append(len(all_nodes) + 1)
                    jobs['allocated_gpus_real'].append({len(all_nodes): [i for i in range(value['gpu'])]})
                    #break
                    continue
                
                if job_pods[key] not in pod_nodes:
                    jobs['instance_type'].append("unknown")
                    jobs['allocated_gpus'].append({})
                    jobs['node_index'].append(None)
                    jobs['allocated_gpus_real'].append({1: []})
                    break
                
                if job_pods[key] in pod_nodes:
                    jobs['instance_type'].append(node_instances[pod_nodes[job_pods[key]]])
                else:
                    jobs['instance_type'].append("unknown")

                if job_pods[key] in pod_nodes:
                    jobs['allocated_gpus'].append({nodes[pod_nodes[job_pods[key]]]: []})
                    jobs['node_index'].append(nodes[pod_nodes[job_pods[key]]])
                else:
                    jobs['allocated_gpus'].append({})
                    jobs['node_index'].append(None)
                
                if job_pods[key] in pod_nodes:
                    jobs['node'].append(pod_nodes[job_pods[key]])
                else:
                    jobs['node'].append("unknown")
            
                if 'gpu_index' in cluster:
                    gpu_index = cluster['gpu_index'][job_pods[key]]
                    gpu_pod = job_pods[key]
                    gpu_index = gpu_index.partition("||")[0]
                    gpu_index = gpu_index.split("\n")
                    
                    for index in gpu_index:
                        if index != "":
                            gpu_node[index] = pod_nodes[gpu_pod]
                            if pod_nodes[gpu_pod] not in node_gpus:
                                node_gpus[pod_nodes[gpu_pod]] = {}
                            if index not in node_gpus[pod_nodes[gpu_pod]]:
                                node_gpus[pod_nodes[gpu_pod]][index] = len(node_gpus[pod_nodes[gpu_pod]])

                        if index != "" and index not in gpu_indices:
                            # TODO: Clean up later
                            gpu_indices[index] = gpu_num
                            gpu_num += 1
                    gpu_index = [node_gpus[pod_nodes[gpu_pod]][index] for index in gpu_index if index]
                    jobs['allocated_gpus_real'].append({nodes[pod_nodes[job_pods[key]]]: gpu_index})
                else:
                    jobs['allocated_gpus_real'].append({nodes[pod_nodes[job_pods[key]]]: []})
            except Exception as e:
                import traceback
                trace = traceback.format_exc()
                print(trace)
                if cluster_type == "cloud":
                    print('cloud')
                else:
                    print('onprem')
                
    if not jobs['arrival']:
        print("No job arrival times logged!")
    
    #print(f'nodes {nodes}')
    print(jobs)
    modified_arrival = [a for a in jobs['arrival'] if a]
    #min_arrival = min(modified_arrival)
    min_arrival = min(jobs['arrival'])
    #print(f'minarrival {min_arrival}')
    modified_submission = [s for s in jobs['submission_time'] if s]
    #min_submission = min(modified_submission)
    min_submission = min(jobs['submission_time'])
    #print(f'minsubmission {min_submission}')
    min_arrival = min_submission
    #min_arrival = min(jobs['submission_time'])
    jobs['arrival'] = [i - min_arrival for i in jobs['arrival']]
    jobs['submission_time'] = [i - min_arrival for i in jobs['submission_time']]
    jobs['start'] = [i - min_arrival for i in jobs['start']]
    #jobs['arrival'] = np.array(jobs['arrival'])
    jobs['num_gpus'] =  np.array(jobs['num_gpus'])
    
    hyperparameters = submission_df['hyperparameters']

    for k, v in hyperparameters.items():
        jobs[k] = v

    sweep_metrics = sweep_df[str(run_id)]
    jobs["varying_values"] = sweep_df["varying_values"].keys()
    jobs["fixed_values"] = sweep_df["fixed_values"].keys()

    for k, v in sweep_metrics.items():
        jobs[k + "_sweep"] = v
    
    with open("wait" + str(submission_df['hyperparameters']['wait_time']) + "uni" + str(submission_df['hyperparameters']['uniform_arrival']) + ".pkl", 'wb') as file:
        pickle.dump(jobs, file)

    return jobs, len(all_nodes), hyperparameters

def parse_jobs_df(event_number=logs, avoid_congestion=False):
    #test = log_jobs.retrieve_events_df(event_number=logs, avoid_congestion=False, only_dict=False)
    #print(test)
    #print(len(test))
    #TODO: Move log_jobs from local to remote repo
    events_dfs, sweep_df = log_jobs.retrieve_events_df(event_number=logs, avoid_congestion=False, only_dict=False)
    #cluster_event_data_df, submission_data_df, sweep_data_df
    columns=['idx', 'runtime', 'arrival', 'num_gpus', 'allocated_gpus', 'allocated_gpus_real', 'start', 'instance_type', 'node_index', 'node', 'cpus', 'submission_time', 'wait_times', 'is_local']
    runs = {}
    runs_list = []
    for run_id, events_df in events_dfs.items():
        cluster_event_df, submission_df, cloud_log_list = events_df
        display(submission_df)
        estimated_runtimes = []
        for job in submission_df:
            if job == 'hyperparameters':
                continue
            #estimated_runtimes.append((job, submission_df[job]["job_duration"]))
            estimated_runtimes.append(submission_df[job]["job_duration"])
            #print(job)
            #print(submission_df[job]["job_duration"])
        print(estimated_runtimes)
        onprem_df, cloud_df = cluster_event_df
        try:
            run, _, _ = parse_job_df(cluster_event_df=cluster_event_df, submission_df=submission_df, sweep_df=sweep_df, avoid_congestion=False, columns=columns, run_id=run_id, cloud_log_list=cloud_log_list)
            runs[run_id] = pd.Series(run)
        except Exception as e:
            import traceback
            trace = traceback.format_exc()
            print(trace)
    runs_df = pd.DataFrame.from_dict(runs)
    runs_df = runs_df.transpose()
    return runs_df

jobs_df = parse_jobs_df(event_number=logs, avoid_congestion=False)
#jobs_df = jobs_df[0:2]
display(jobs_df)

In [ ]:
if 'workload_type' in jobs_df: 
    print(jobs_df['workload_type'][0])
print(len(jobs_df['runtime'][0]))
print(len(jobs_df['is_local'][0]))
print(sum(jobs_df['is_local'][0]))
print(len(jobs_df['arrival'][0]))
print(len(jobs_df['start'][0]))
print(len(jobs_df['submission_time'][0]))
print(len(jobs_df['runtime'][0]))
print(len(jobs_df['node'][0]))
print(len(jobs_df['node_index'][0]))
print(len(jobs_df['instance_type'][0]))
print(len(jobs_df['allocated_gpus'][0]))
print(len(jobs_df['allocated_gpus_real'][0]))

print(sorted(jobs_df['runtime'][0]))
print(jobs_df['is_local'][0])
print(jobs_df['arrival'][0])
print(jobs_df['start'][0])
print(jobs_df['submission_time'])
print(jobs_df['runtime'])
print(jobs_df['node_index'])
print(jobs_df['node'][0])
print(jobs_df['instance_type'][0])
print(jobs_df['allocated_gpus'][0])
print(jobs_df['allocated_gpus_real'][0])

In [ ]:
#jobs_df['arrival_mask'] = jobs_df['start'].apply(lambda x: [0 if not i else 1 for i in x])
jobs_df['arrival_mask'] = jobs_df['is_local']#.apply(lambda x: [0 if not i else 1 for i in x])
jobs_df['onprem_mask'] = jobs_df['arrival_mask']

# CLIP WAITS

def cloud_wait_unclipped(row): 
    cloud_wait = [row['wait_times'][i] for i, n in enumerate(row['arrival_mask']) if n == 0]
    return cloud_wait
jobs_df['cloud_wait_unclipped'] = jobs_df.apply(cloud_wait_unclipped, axis=1)

def clipped_wait(row): 
    k8s_scheduling_waiting_constant = 1 #wait_time_2
    onprem_wait = [row['wait_times'][i] for i, n in enumerate(row['arrival_mask']) if n == 1]
    cloud_wait = [k8s_scheduling_waiting_constant + row['wait_time'] for i, n in enumerate(row['arrival_mask']) if n == 0]
    new_wait = onprem_wait + cloud_wait
    return new_wait

jobs_df['wait_times'] = jobs_df.apply(clipped_wait, axis=1)

def cloud_wait(row): 
    k8s_scheduling_waiting_constant = 1 #wait_time_2
    cloud_wait = [k8s_scheduling_waiting_constant + row['wait_time'] for i, n in enumerate(row['arrival_mask']) if n == 0]
    return cloud_wait
jobs_df['cloud_wait'] = jobs_df.apply(cloud_wait, axis=1)


def onprem_wait(row): 
    k8s_scheduling_waiting_constant = 1 #wait_time_2
    onprem_wait = [row['wait_times'][i] for i, n in enumerate(row['arrival_mask']) if n == 1]
    return onprem_wait

jobs_df['onprem_wait'] = jobs_df.apply(onprem_wait, axis=1)

# COMPUTE METRICS

jobs_df['avg_wait'] = jobs_df['wait_times'].apply(lambda x: sum(x)/len(x))
jobs_df['avg_runtime'] = jobs_df['runtime'].apply(lambda x: sum(x)/len(x))

def compute_total_time(row):
    total_time = [row['wait_times'][i] + row['runtime'][i] for i in range(len(row['wait_times']))]
    return total_time


jobs_df['total_time'] = jobs_df.apply(compute_total_time, axis=1)

def compute_completion_time(row):
    # TODO: Submission time is time submitted to starburst 
    #completion_time = [row['total_time'][i] + row['submission_time'][i] for i in range(len(row['wait_times']))]
    completion_time = [row['total_time'][i] + row['arrival'][i] for i in range(len(row['wait_times']))]
    return completion_time

jobs_df['completion_time'] = jobs_df.apply(compute_completion_time, axis=1)
#jobs_df['avg_jct'] = jobs_df['total_time'].apply(lambda x: sum(x)/len(x))

def compute_avg_jct(row):
    arrival_times = row['arrival']
    run_time = row['runtime']
    wait_time = row['wait_times']
    resources = row['num_gpus']
    starts = row['start']
    idxes = row['idx']
    is_locals = row['is_local']
    sort_zip = sorted(zip(idxes, arrival_times, run_time, starts, resources, wait_time))
    
    #start = sort_zip[10][1]
    #end = sort_zip[-10][1]
    
    total_used_space = 0
    total_time = 0
    
    for l in sort_zip:
        #job_idx = l[0]
        #job_arrival = l[1]
        job_runtime = l[2]
        job_wait_time = l[1]
        #job_start = l[3]
        job_gpus = l[4]
        wait_time = l[5]
        #inter_start = max(job_start, start)
        #inter_end = min(job_start + job_runtime, end)
        #if inter_end >= inter_start:
        #    total_used_space+= job_gpus * (inter_end - inter_start)
        total_time += job_runtime
        total_time += wait_time
        
    jct = total_time/len(sort_zip)#total_used_space/(row['cluster_size']*4*row['gpus_per_node']*(end-start))
    jct = jct/3600
    return jct
jobs_df['avg_jct'] = jobs_df.apply(compute_avg_jct, axis=1)

def compute_cluster_utilization(row):
    arrival_times = row['arrival']
    run_time = row['runtime']
    resources = row['num_gpus']
    starts = row['start']
    idxes = row['idx']
    is_locals = row['is_local']
    sort_zip = sorted(zip(idxes, arrival_times, run_time, starts, resources, is_locals))
    
    start = sort_zip[0][1]
    end = sort_zip[-1][1]
    
    total_used_space = 0
    for l in sort_zip:
        job_idx = l[0]
        job_arrival = l[1]
        job_runtime = l[2]
        job_start = l[3]
        job_gpus = l[4]
        is_local = l[5]
        if is_local==1:
            inter_start = max(job_start, start)
            inter_end = min(job_start + job_runtime, end)
            #import pdb; pdb.set_trace()
            if inter_end >= inter_start:
                total_used_space+= job_gpus * (inter_end - inter_start)
    cluster_utilization = total_used_space/(row['cluster_size']*4*row['gpus_per_node']*(end-start))
    #import pdb; pdb.set_trace()
    return cluster_utilization
        
#     # TODO: Remove previous arrival and 
#     print(row['arrival'])
#     surface_area = [row['runtime'][i] * row['num_gpus'][i] for i in range(len(row['runtime']))]
#     utilized_surface_area = sum(surface_area)
#     total_surface_area = (max(row['completion_time']) - min(row['submission_time'])) * (row['gpus_per_node'] * 1) #row['cluster_size'])
#     cluster_utilization = utilized_surface_area/total_surface_area
#     return cluster_utilization

jobs_df['cluster_utilization'] = jobs_df.apply(compute_cluster_utilization, axis=1)

def compute_system_utilization(row):
    arrival_times = row['arrival']
    run_time = row['runtime']
    resources = row['num_gpus']
    starts = row['start']
    idxes = row['idx']
    is_locals = row['is_local']
    sort_zip = sorted(zip(idxes, arrival_times, run_time, starts, resources, is_locals))
    
    start = sort_zip[0][1]
    end = sort_zip[-1][1]
    
    total_used_space = 0
    for l in sort_zip:
        job_idx = l[0]
        job_arrival = l[1]
        job_runtime = l[2]
        job_start = l[3]
        job_gpus = l[4]
        is_local = l[5]
        inter_start = max(job_start, start)
        inter_end = min(job_start + job_runtime, end)
        if inter_end >= inter_start:
            total_used_space += job_gpus * (inter_end - inter_start)
    system_utilization = total_used_space/(row['cluster_size']*4*row['gpus_per_node']*(end-start))
    
    return system_utilization

jobs_df['system_utilization'] = jobs_df.apply(compute_system_utilization, axis=1)

        
#     # TODO: Remove previous arrival and 
#     print(row['arrival'])
#     surface_area = [row['runtime'][i] * row['num_gpus'][i] for i in range(len(row['runtime']))]
#     utilized_surface_area = sum(surface_area)
#     total_surface_area = (max(row['completion_time']) - min(row['submission_time'])) * (row['gpus_per_node'] * 1) #row['cluster_size'])
#     cluster_utilization = utilized_surface_area/total_surface_area
#     return cluster_utilization


#def compute_system_utilization(row):
    # TODO: Compute this value correctly
    #return system_utilization

#jobs_df['cluster_utilization'] = jobs_df.apply(compute_cluster_utilization, axis=1)
GCP_PRICES = {
    "e2-medium": 0.038795,
    "e2-standard-8": 0.31036,
    "unknown": 0.038795,
    "n1-standard-96": 4.56 
}


def compute_total_cost(row):
    run_time = row['runtime']
    resources = row['num_gpus']
    idxes = row['idx']
    is_locals = row['is_local']
    sort_zip = sorted(zip(idxes, run_time, resources, is_locals))

    
    total_cloud_cost = 0
    for l in sort_zip:
        job_idx = l[0]
        job_runtime = l[1]
        job_gpus = l[2]
        is_local = l[3]
        if is_local==0:
            total_cloud_cost += job_runtime * job_gpus

    return total_cloud_cost/3600
    # TODO: Compute this value correctly
    # Get all cloud runtimes + submit 
#     total_time = [row['runtime'][i] * row['num_gpus'][i] * (1 - row['arrival_mask'][i]) * GCP_PRICES[row['instance_type'][i]] for i in range(len(row['arrival_mask']))]
#     return sum(total_time)

jobs_df['total_cloud_cost'] = jobs_df.apply(compute_total_cost, axis=1)

In [ ]:
jobs_df
jobs_df['node'][0]

In [ ]:
def index_mapping(jobs=None, gpus_per_node=8, resource='cpu'):#gpu_value=None):
    '''
    Implement greedly algorithm with heap to place jobs to free resource indicies
    
    Specify which resource (e.g. gpu, cpu) to use fitting jobs to indices
    
    (1) Add all jobs to queue, then greedily assign indicies 
    (2) Have priority queue for each node with "Free indices" sorted by index number 
    (3) Iterate over all start times 

    TODO: Fix 1-off CPU index error 
    TODO: Don't let any job_idx values to be -1, and ensure it starts at node 1
    TODO: Verify allocated_nodes
    TODO: Determine how parsing changes between http_info and default values 
    TODO: Verify arrival value is accurate
    TODO: Create a list of times that include all arrival times and completion times in the same list in numerical order 
    '''
    '''
    if gpu_jobs: 
        gpu_value = 'allocated_gpus_index'
    else: 
        gpu_value = 'allocated_gpus'
    '''
    workload = 'gpu'
    if 'workload_type' in jobs: 
        print('workload type')
        print(jobs['workload_type'])
        workload = jobs['workload_type']
    print('gpus')
    print(jobs['num_gpus'])
    print('cpus list')
    print(jobs['cpus'])
    #import pdb; pdb.set_trace()
    GPUS_PER_NODE = gpus_per_node
    allocated_nodes = jobs['node_index']
    
    
    #if resource == 'gpu':
    if workload == 'gpu':
        cpus = jobs['num_gpus']
    else: 
        cpus = jobs['cpus']
    #gpus = jobs['gpus']
    nodes = set(allocated_nodes)
    
    node_jobs ={}
    node_queues = {}
    for node in nodes:
        node_queues[node] = [i + 1 for i in range(GPUS_PER_NODE)]
        node_jobs[node] = []

    global_queue = [] # Queue sorted on end time -- earliest to latest end time
    job_id_to_index = {} 

    for i in range(len(jobs['arrival'])):
        #Remove values from queue
        #import pdb; pdb.set_trace()
        job_id = jobs['idx'][i]
        job_id_to_index[job_id] = i
        job_node = jobs['node_index'][i]
        #if resource == 'gpu':
        if workload == 'gpu':
            job_cpu_size = jobs['num_gpus'][i]
        else: 
            job_cpu_size = jobs['cpus'][i]
        job_arrival = jobs['arrival_plot'][i]
        job_runtime = jobs['runtime'][i]
        
        while global_queue and global_queue[0][0] <= job_arrival: 
            end_time, end_job_id = heapq.heappop(global_queue)
            released_index = job_id_to_index[end_job_id]
            for released_node in jobs['allocated_gpus'][released_index]: 
                released_cpus = jobs['allocated_gpus'][released_index][released_node]
                released_node_queue = node_queues[released_node]
                try:
                    node_jobs[released_node].remove(end_job_id)
                except:
                    #import pdb; pdb.set_trace()
                    print("Job Id not found")
                    print(end_job_id)
                    print(node_jobs[released_node])
                    continue
                for cpu in released_cpus:
                    heapq.heappush(released_node_queue, cpu)

        heapq.heappush(global_queue, (job_arrival + job_runtime, job_id))
        job_allocated_cpus = []
        node_queue = node_queues[job_node]
        node_jobs[job_node].append(job_id)
    
        print(f'job_cpu_size {job_cpu_size}')
        try:
            for j in range(job_cpu_size):
                cpu_index = heapq.heappop(node_queue)
                job_allocated_cpus.append(cpu_index)
        except:
            print("not enough cpus to fit jobs")
            print(node_queue)
            print(job_allocated_cpus)
        
        print(f'allocated resources -- cpus or gpus: {job_allocated_cpus}')

        jobs['allocated_gpus'][i] = {job_node: job_allocated_cpus}
        #print(jobs['allocated_gpus'][i])
        #import pdb; pdb.set_trace()

    return jobs

def generate_gantt_chart(row=None, gpus_per_node=None, ratio=None, scale=None, gpu_jobs=None):
    '''
    Create "threads index" that track CPU jobs running together
    #TODO: Modify function to plot CPU jobs --> number of jobs concurrently running may exceed cpu count
    #TODO: Plot color based on job start time and not job index
    #TODO: Determine why jobs dissapaear when strings labels are used for nodes
    #TODO: Label each row of jobs with the name of the node -- not just integers 
    #TODO: Plot cloud values in a separate plot
    '''
    workload = "gpu"
    if 'workload_type' in row: 
        workload = row['workload_type']
        
    #if 'workload_type' in row and row['workload_type'] == 'cpu': 
    if workload == 'cpu':
        gpus_per_nodes = 2
        
    graphs = ['cloud', 'onprem']
    for graph in graphs:
        #import pdb; pdb.set_trace()
        save=False; path=None; subplt=None; plt_index=None; tag=None; plot_sweep=False
        varying_values = row['varying_values']
        tag = ""

        count = 0 
        for value in varying_values: 
            tag += str(value)
            tag += ":"
            if isinstance(row[value], str) and len(row[value]) < 50: 
                tag += str(row[value])
            tag += " | "
            count += 1
            if count % 5 == 0: 
                tag += '\n'
                
        gpu_value = 'allocated_gpus'#_real'
        
        '''
        if 'workload_type' in row and row['workload_type'] == 'cpu': 
            gpu_value = 'allocated_gpus'
        elif gpu_jobs: 
            gpu_value = 'allocated_gpus_real'
        else: 
            gpu_value = 'allocated_gpus'
        '''

        fig, ax = plt.subplots()

        #GPUS_PER_NODE = row['cpus_per_node']
        GPUS_PER_NODE = gpus_per_node
        
        '''
        if 'workload_type' in row: 
            print('workload type')
            print(row['workload_type'])
            row = index_mapping(row, GPUS_PER_NODE, row['workload_type'])
        else: 
            row = index_mapping(row, GPUS_PER_NODE, 'gpu')
        '''
        
        row = index_mapping(row, GPUS_PER_NODE, 'gpu')
        
        '''
        if 'workload_type' in row and row['workload_type'] == 'cpu':
            print("mapped indices")
            row = index_mapping(row, GPUS_PER_NODE, 'cpu')
        #if not gpu_jobs: 
        elif graph == 'cloud':
            GPUS_PER_NODE = 32
            #print(row['allocated_gpus'])
            row = index_mapping(row, GPUS_PER_NODE, 'gpu')
            #print(row['allocated_gpus'])
        #import pdb; pdb.set_trace()
        #if row['workload_type'] == 'cpu':
        '''

        
        #print("allocated_gpus_real")
        #print(row['allocated_gpus_real'])
        print("allocated_gpus")
        print(row['allocated_gpus'])
        
        NUM_COLORS = len(row['idx'])
        cm = plt.get_cmap('gist_rainbow')
        colors = [cm(1. * i / NUM_COLORS) for i in range(NUM_COLORS)]

        y_lim_min = 1000
        y_lim_max = -1000
        num_nodes = row['cluster_size'] + row['cloud_cluster_nodes']

        total_gpus = num_nodes * GPUS_PER_NODE #GPUs equivalent to CPUs -- if no GPU's then GPUS_PER_NODE reflects cpus
        segment_height_list = {}
        gpu_indices = {}
        gpu_rows = set()
        node_name = ""

        # TODO: Plot infinite cloud spillover 
        try: 
            #import pdb; pdb.set_trace()
            for j_idx in range(len(row['idx'])):
                allocated_gpus = row[gpu_value][j_idx]
                
                #if graph == 'cloud':
                #    allocated_gpus = row['allocated_gpus'][j_idx]
                
                #print(allocated_gpus)
                #print(allocated_gpus)
                #segment = (row['arrival'][j_idx],
                #            row['arrival'][j_idx] + row['runtime'][j_idx], j_idx)
                #segment = (row['submission_time'][j_idx],
                #            row['submission_time'][j_idx] + row['runtime'][j_idx], j_idx)
                segment = (row['arrival_plot'][j_idx],
                            row['arrival_plot'][j_idx] + row['runtime'][j_idx], j_idx)

                node_name = row['node'][j_idx]
                if graph == 'onprem' and node_name == 'cloud': 
                    continue
                elif graph == 'cloud' and node_name != 'cloud':
                    continue
                
                print(f'j_idx {j_idx}')
                
                #if graph == 'cloud' and node_name == 'cloud':
                #    print(allocated_gpus)
                
                for node_idx in allocated_gpus.keys():
                    for node_gpu_idx in allocated_gpus[node_idx]:
                        if graph == 'cloud': 
                            gpu_idx = node_gpu_idx
                            #print(gpu_idx)
                        else: 
                            gpu_idx = total_gpus - (GPUS_PER_NODE * node_idx + node_gpu_idx)
                        print(f'gpu_idx {gpu_idx}')
                        
                        gpu_rows.add(gpu_idx)
                        #print(node_gpu_idx)
                        #print(gpu_rows)
                        gpu_indices[node_name] = [gpu_idx]
                        y_lim_min = min(y_lim_min, gpu_idx) #- 8)#if gpu_idx > 0 else gpu_index - 8)
                        
                        y_lim_max = max(y_lim_max, gpu_idx + 1) #+ 8)
                        if graph == 'cloud':
                            y_lim_max = GPUS_PER_NODE

                        plt.barh(gpu_idx,
                                    width=row['runtime'][j_idx],
                                    edgecolor='black',
                                    height=1.0,
                                    left=segment[0],
                                    align='edge',
                                    color=colors[row['idx'][j_idx]] if row['idx'][j_idx] < len(colors) else None,
                                    alpha = 0.5)    
        except Exception as e:
            import traceback
            t = traceback.format_exc()
            print(t)
            print(e)

        for i in range(total_gpus + 1):
            multiplier = math.ceil(num_nodes / 32)
            if (i + 1) % GPUS_PER_NODE == 1:
                plt.axhline(y=i + 1, linewidth=3 / multiplier, color='black')
            else:
                pass

        max_arrival = max(row['arrival_plot'])
        completions = [row['arrival_plot'][i] + row['runtime'][i] for i in range(len(row['arrival_plot']))]
        max_completion = max(completions)

        x_lim_max = max_completion
        last_job_time = max(row['completion_time'])
        #print(last_job_time)
        #26913.0
        #18405.78506708145
        last_job_time= 1000#18405.78506708145#26913
        dim=(y_lim_min, y_lim_max, 0, last_job_time)
        bottom, top, left, right = dim
        plt.ylim(bottom=bottom, top=top)
        plt.xlim(left=left, right=right)
        plt.axvline(x=max_arrival, color='brown', linewidth=0.75)
        plt.xlabel('Time (hrs)')
        plt.ylabel('Nodes ')
        #if graph == 'cloud'
        plt.title(str(tag))
        if graph == 'cloud':
            plt.title(f'Cloud {workload} Jobs')
        elif graph == 'onprem':
            plt.title(f'Onprem {workload} Jobs')
        #plt.title(graph)

        gpu_labels = sorted([(v, k) for k, v in gpu_indices.items()])
        ticks = [label[0] for label in gpu_labels]
        ticks = np.array(ticks)
        ticks = ticks.flatten()
        labels = [label[1] for label in gpu_labels]
        labels = np.array(labels)
        labels = labels.flatten()
        plt.yticks(ticks)
        labels = [i for i in range(len(labels))]
        ax.set_yticklabels(labels)
        new_labels = {}
        node_count = 0
        #for k, v in labels.items():
            #new_labels[node_count] = k #node_count
            #node_count += 1
            #new_labels.append(node_count)
        #labels = {1.0: 'High', 0.0: 'Medium', -1.0: 'Low'}
        #ax.set_yticks(list(labels.keys()))
        #ax.set_yticks(list(new_labels.keys()))
        plt.rcParams.update({'font.size': 20})
        plt.tick_params(axis='y', which='both', bottom=False, top=False, labelbottom=False, right=False, left=False, labelleft=False)
        ax.set_ylabel('')
        
        def divide(x, pos):
            return '{}'.format(round(x / 3600, 1))

        # Set the formatter
        import matplotlib.ticker as ticker
        formatter = ticker.FuncFormatter(divide)
        ax.xaxis.set_major_formatter(formatter)
        
        if save:
            if path: 
                plt.savefig(path)
                plt.close()
        else:
            plt.show()
        
def remap_cloud_arrival(row):
    new_arrival = [row['start'][i] if row['is_local'][i] == 1 else row['arrival'][i] for i in range(len(row['arrival']))]
    #new_arrival = [1 if row['is_local'] == 1 else 0 for i in range(len(row['arrival']))]
    return new_arrival

jobs_df['arrival_plot'] = jobs_df.apply(remap_cloud_arrival, axis=1)
print(list(enumerate(jobs_df['arrival_plot'][0])))
#print(jobs_df['is_local'][0])
print('start times')
print(jobs_df['start'][0])
print(jobs_df['submission_time'][0])
schedule_time = [jobs_df['start'][0][i] - jobs_df['submission_time'][0][i] for i in range(len(jobs_df['start'][0]))]
print("pod scheduling time (seconds)")
print(schedule_time)
print("run locally (1=local, 0=cloud)")
print(jobs_df['is_local'][0])
print("allocated_gpus_real")
print(jobs_df['allocated_gpus_real'][0])
print("allocated_gpus")
print(jobs_df['allocated_gpus'][0])
print(jobs_df['arrival'][0])
print("IDs")
print(jobs_df['idx'][0])
print("RUNTIME")
print(jobs_df['runtime'][0])
print("VERIFYING")
print(sorted(jobs_df['arrival'][0]))

for j in range(len(jobs_df)):
    cloud_time = [jobs_df['runtime'][j][i] for i in range(len(jobs_df['runtime'][j])) if jobs_df['is_local'][j][i] == 0  ]
    print("cloud runtimes (seconds)")
    print(cloud_time)
    onprem_time = [jobs_df['runtime'][j][i] for i in range(len(jobs_df['runtime'][j])) if jobs_df['is_local'][j][i] == 1  ]
    print("onprem runtimes (seconds)")
    print(onprem_time)

for j in range(len(jobs_df)):
    total_job_arrival = len(jobs_df['arrival'][j])
    total_job_submission = len(jobs_df['start'][j])
    total_job_completion = len(jobs_df['completion_time'][j])
    print(f'{j} {total_job_arrival} {total_job_submission} {total_job_completion}')
gpus_per_node = 8 #TODO: Parse this value from the job
ratio = (1, 1)
scale = 1

jobs_df.apply(generate_gantt_chart, axis=1, args=(gpus_per_node, ratio, scale, True))

In [ ]:
def parse_starburst_log(event_number=logs):
    log = log_jobs.retrieve_log(event_number=logs)
    return log 

log_df = parse_starburst_log(event_number=logs)
log_df

loop_times = []
interloop_times = []
process_queue_times = []
process_event_times = []
queue_add_times = []
await_times = []
times = []
events = []

for i in range(len(log_df)):
    log = log_df[0][i]
    parts = log.split('||')
    if len(parts) > 1:
        log = parts[1]
        
    if log.find("TICK TIME (())") != -1:
        parts = log.split(' ')
        if len(parts) > 8: 
            time = parts[4]
            times.append(float(time))
            eventtype = parts[6] + parts[7]
            events.append(eventtype)
            
    if log.find("LOOP TIME (())") != -1:
        parts = log.split(' ')
        time = parts[4]
        loop_times.append(float(time))
        
    if log.find("INTERLOOP TIME (())") != -1:
        parts = log.split(' ')
        time = parts[4]
        interloop_times.append(float(time))
        
    if log.find("PROCESSQUEUE TIME (())") != -1:
        parts = log.split(' ')
        time = parts[4]
        process_queue_times.append(float(time))
        
    if log.find("PROCESSEVENT TIME (())") != -1:
        parts = log.split(' ')
        time = parts[4]
        process_event_times.append(float(time))
    
    if log.find("QUEUEADD TIME (())") != -1:
        parts = log.split(' ')
        time = parts[4]
        queue_add_times.append(float(time))
        
    if log.find("AWAIT TIME (())") != -1:
        parts = log.split(' ')
        time = parts[4]
        await_times.append(float(time))

fig, axs = plt.subplots(nrows=1, ncols=9,figsize=(12,3)) 

#loop_times = [loop_times[i + 1] - loop_times[i] for i in range(len(loop_times) - 1)]

overflows = []
addtimes = []
ticktimes = []
for i in range(len(times)):
    time = times[i]
    event = events[i]
    if event == 'JobAddEvent:Job,':
        addtimes.append(time)
    if event == 'SchedTick,':
        ticktimes.append(time)
    if time > 0.6: 
        overflows.append((time, event))

axs[0].set_title('All Event Times', fontsize =10)
axs[0].hist(times)
axs[0].set_xlabel('Time (sec)')
axs[0].set_ylabel('Frequency')

axs[1].set_title('Job Add Event Times', fontsize =10)
axs[1].hist(addtimes)
axs[1].set_xlabel('Time (sec)')
axs[1].set_ylabel('Frequency')

axs[2].set_title('Sched Tick Event Times', fontsize =10)
axs[2].hist(ticktimes)
axs[2].set_xlabel('Time (sec)')
axs[2].set_ylabel('Frequency')

axs[3].set_title('Loop Times', fontsize =10)
axs[3].hist(loop_times)
axs[3].set_xlabel('Time (sec)')
axs[3].set_ylabel('Frequency')

axs[4].set_title('Inter Loop Times', fontsize =10)
axs[4].hist(interloop_times)
axs[4].set_xlabel('Time (sec)')
axs[4].set_ylabel('Frequency')

axs[5].set_title('Process Queue Times', fontsize =10)
axs[5].hist(process_queue_times)
axs[5].set_xlabel('Time (sec)')
axs[5].set_ylabel('Frequency')

axs[6].set_title('Process Event Times', fontsize =10)
axs[6].hist(process_event_times)
axs[6].set_xlabel('Time (sec)')
axs[6].set_ylabel('Frequency')

axs[7].set_title('Queue Add Times', fontsize =10)
axs[7].hist(queue_add_times)
axs[7].set_xlabel('Time (sec)')
axs[7].set_ylabel('Frequency')

axs[8].set_title('Await Times', fontsize =10)
axs[8].hist(queue_add_times)
axs[8].set_xlabel('Time (sec)')
axs[8].set_ylabel('Frequency')

print(process_event_times)
print(process_queue_times)
print(loop_times)
print(interloop_times)
print(queue_add_times)
print(sum(loop_times)/len(loop_times))

add_count = 0 
sched_count = 0
for t, e in overflows: 
    if e == 'JobAddEvent:Job,':
        add_count += 1
    if e == 'SchedTick,':
        sched_count += 1

#print(add_count)
#print(sched_count)

In [ ]:
for i in range(len(jobs_df)):
    print(i)
    print(len(jobs_df['is_local'][i]))
    print(sum(jobs_df['is_local'][i]))
    print(len(jobs_df['arrival'][i]))
    print(len(jobs_df['start'][i]))
    print(len(jobs_df['submission_time'][i]))
    print(len(jobs_df['runtime'][i]))
    print(len(jobs_df['allocated_gpus_real'][i]))

In [ ]:
jobs_df
#print(len(jobs_df['idx'][0]))
'''
print(jobs_df['is_local'][0][:-18])
print(jobs_df['arrival'][0][:-18])
print(jobs_df['start'][0][:-18])
print(jobs_df['submission_time'][0][:-18])
print(jobs_df['runtime'][0][:-18])
print(jobs_df['allocated_gpus_real'][0][:-18])
'''
print(len(jobs_df['is_local'][0]))
print(len(jobs_df['arrival'][0]))
print(len(jobs_df['start'][0]))
print(len(jobs_df['submission_time'][0]))
print(len(jobs_df['runtime'][0]))
print(len(jobs_df['allocated_gpus_real'][0]))

In [ ]:
print(jobs_df['total_cloud_cost'])
print(jobs_df['avg_jct'])
print(jobs_df['cluster_utilization'])

jobs_df['completion_time']

In [ ]:
for i in jobs_df['runtime']:
    print(i)
    print(len(i))

for i in jobs_df['arrival_mask']:
    print(i)
    print(len(i))
    
for i in jobs_df['onprem_mask']:
    print(i)
    print(len(i))
    
for i in jobs_df['num_gpus']:
    print(i)
    print(len(i))
    
for i in jobs_df['start']:
    print(i)
    print(len(i))
    
for i in jobs_df['instance_type']:
    print(i)
    print(len(i))

In [ ]:
def wait_time_histograms(row):
    """Plots waiting time for onprem, cloud, and cloud clipped"""
    #import pdb; pdb.set_trace()
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(12,3)) 
    
    axs[0].set_title("ONPREM " + "wait " + str(row['wait_time']) +  " arr " + str(row['arrival_rate']), fontsize=10)
    axs[0].hist(row['onprem_wait'])
    if row['onprem_wait'] != []:
        average = sum(row['onprem_wait'])/len(row['onprem_wait'])
        axs[0].axvline(average, color='r', linewidth=0.5)

    axs[1].set_title("CLOUD " + "wait " + str(row['wait_time']) +  " arr " + str(row['arrival_rate']), fontsize=10)
    axs[1].hist(row['cloud_wait'])
      
    axs[2].set_title("CLOUD UNCLIPPED " + "wait " + str(row['wait_time']) +  " arr " + str(row['arrival_rate']), fontsize=10)
    axs[2].hist(row['cloud_wait_unclipped'])
    
    plt.show()

jobs_df.apply(wait_time_histograms, axis=1)

In [ ]:
jobs_df
jobs_df['allocated_gpus_real'][0]

In [ ]:
job_uuid = list(jobs_df['idx'])[0]
job_runtimes = list(jobs_df['runtime'])[0]
job_gpus = list(jobs_df['num_gpus'])[0]
job_gpus = [int(i) for i in job_gpus]
print(job_uuid)
print(job_runtimes)
print(job_gpus)
print(sum(job_runtimes)/len(job_runtimes))

In [ ]:
def set_plotting_setting(ax):
    plt.rcParams.update({'font.size': 15})
    ax.grid(True, which='both')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(True)
    ax.spines['bottom'].set_visible(True)
    #ax.tick_params(bottom=False, left=False)
    ax.tick_params(bottom=True, left=False)
    ax.set_axisbelow(True)

label_dict = {
    'avg_jct': 'Avg. JCT (sec)',
    'cost_mult': '% Cost Savings\nover No Wait',
    'cost_diff': 'Cost Savings\nover No Wait',
    'cluster_size': 'Cluster Size (# Nodes)',
    'norm_system_utilization': 'System Utilization',
    'system_utilization': 'System Utilization',
    'cluster_utilization': 'Cluster Utilization',
    'total_cloud_cost': 'Cloud Cost',
    'arrival_rate': 'Arrival Rate',
    'uniform_arrival': 'Uniform Inter Arrival Time',
}

legend_dict = {
    'constant': 'Constant',
    'linear_runtime': 'Runtime',
    'linear_cost': 'Cost',
    'zero': 'No Wait',
    'linear_runtime_filter_cpu': 'Runtime-Preempt-CPU'
}

def simulator_plotting_fn(df, 
                          x_axis: str,
                          y_axis: list = ['cost_mult', 'avg_jct'],
                          df_filter: dict = {},
                          baseline_filter: dict={'waiting_policy': 'zero',},
                          groupby_values=['waiting_policy', 'waiting_factor'],
                          normalize_x_axis=False,
                          fig_ratio=(5, 3.5),
                          intermediate_df={"baseline_df": False, "merged_df": False, "metrics_df": False, "groups_df": False},
                          return_df=False):
    """
    Takes a baseline filter to plot metrics of different policies against
    
    Creates a product of the baseline filter against each existing run
        - Values ending with _y are values from the baseline
    
    Takes a groupby value to compress to set of useful pairs
    """
    
    def cost_multiplier(row):
        baseline_cost = row['total_cloud_cost_y']
        cost = row['total_cloud_cost_x']
        if baseline_cost == 0 and cost==0:
            return 0
        #elif baseline_cost <=10000:
            # Small cloud cost for No wait
            # Savings over small cloud cost is negligible for organizations.
        #    return 0
        elif baseline_cost == 0 and cost>0:
            return 100
        return 100* (1 - (cost/baseline_cost))
    
    def cost_difference(row):
        baseline_cost = row['total_cloud_cost_y']
        cost = row['total_cloud_cost_x']
        return baseline_cost - cost
    
    
    if isinstance(y_axis, str):
        y_axis = [y_axis]
    fig, axs = plt.subplots(nrows=1, ncols=len(y_axis), figsize=(fig_ratio[0]*len(y_axis), fig_ratio[1]))
    
    if len(y_axis) == 1: 
        if not isinstance(axs, list):
            axs = [axs]
    #print(axs)
    '''
    for k,v in df_filter.items():
        if isinstance(v, list):
            df = df[df[k]==v]
    '''

    for k,v in df_filter.items():        
        mask = df[k].apply(lambda x: x == v)
        if isinstance(v, list):
            df = df[mask]
            
    #TODO: get_default = df after mask 
    baseline_df = df
    if intermediate_df['baseline_df']:
        print("baseline")
        display(baseline_df)
    
    for k,v in baseline_filter.items():
        assert not isinstance(v, list)
        baseline_df = baseline_df[baseline_df[k]==v]
        #df = df[df[k]!=v]
         
    # Merge to check for baseline
    diff_df = pd.merge(df, baseline_df, left_on=x_axis,right_on=x_axis)
    
    #TODO: join_baseline = diff_df after merge
    if intermediate_df['merged_df']:
        print("merged")
        display(diff_df)
    
    if normalize_x_axis:
        if x_axis == 'cluster_size':
            # Hardcoded in Philly trace, precomputed ahead of time
            if df['dataset'].iloc[0] == 'philly':
                total_job_volume = 1155998.77277777
                job_makespan = 2559.3205555555555
            elif df['dataset'].iloc[0] == 'helios':
                total_job_volume = 1853756.1241666232
                job_makespan = 4651.911388888889
            diff_df['norm_system_utilization'] = total_job_volume/(job_makespan*diff_df['cluster_size']*df['gpus_per_node'].iloc[0])
            x_axis = 'norm_system_utilization'
        elif x_axis == 'arrival_rate' or x_axis == 'uniform_arrival':
            # Volume rate => runtime must be in either hours or seconds 
            '''
            Verification: 
            - Arrival_Rate = VERIFIED ~ error 
            - Num_gpus = Verified
            - Runtime = Verified 
            '''
            #arrival_rate = diff_df['arrival_rate'] # Jobs per second 
            arrival_rate = 1/df['uniform_arrival'] # uniform_arrival => time between jobs in seconds 
            #avg_job_volume_rate = arrival_rate * np.mean(df['num_gpus'].iloc[0]* df['runtime'].iloc[0])
            df['volume'] = df['num_gpus'] * df['runtime']
            #df['avg_job_volume_rate'] = sum(df['volume'])/len(df['volume']) * arrival_rate
            df['avg_volume'] = df['volume'].apply(lambda arr: np.mean(arr))
            df['avg_job_volume_rate'] = df['avg_volume'] * arrival_rate
            #import pdb; pdb.set_trace()
            #avg_job_volume_rate = arrival_rate * ([df['num_gpus'] * df['runtime'][i] for i in range(len(df['runtime']))])/len(df['runtime'])
            #df['avg_job_volume_rate'] = avg_job_volume_rate
            df['verify_mean_runtime'] = np.mean(df['num_gpus'].iloc[0]* df['runtime'].iloc[0])
            df['verify_cluster_nodes'] = (df['cluster_size'].iloc[0]*df['gpus_per_node'].iloc[0])
            #print(avg_job_volume_rate)
            #print(df['cluster_size'].iloc[0])
            #print(df['gpus_per_node'].iloc[0])
            #print(df['cluster_size'].iloc[0]*df['gpus_per_node'].iloc[0])
            #diff_df['norm_system_utilization'] = avg_job_volume_rate/(df['cluster_size'].iloc[0]*df['gpus_per_node'].iloc[0])
            #df['cluster_size'] = 1 #TODO: Remove this value
            diff_df['norm_system_utilization'] = df['avg_job_volume_rate']/(df['cluster_size']*df['gpus_per_node'])
            diff_df['norm_system_utilization'] = pd.to_numeric(diff_df['norm_system_utilization'], errors='coerce')
            diff_df['norm_system_utilization'] = diff_df['norm_system_utilization'].round(4)
            x_axis = 'norm_system_utilization'
            diff_df = diff_df.sort_values('norm_system_utilization')
    
    
    diff_df['cost_mult'] = diff_df.apply(cost_multiplier, axis=1)
    diff_df['cost_diff'] = diff_df.apply(cost_difference, axis=1)
    
    if intermediate_df['metrics_df']:
        print("merged metrics")
        display(diff_df)
        
    #TODO: get_baseline = df after normalizing axis
    if groupby_values: 
        groupby_values = [f'{g}_x' for g in groupby_values]
    mod_y_axis = [f'{y}_x' if y!='cost_mult' and y!='cost_diff' else y for y in y_axis]
    
    markers = itertools.cycle(('v', '^','.', 'o', '*',',', '+',)) 
    groups = diff_df.groupby(groupby_values)
    
    if intermediate_df['groups_df']:
        for name, group in groups:
            print(f"Group: {name}")
            #'cost_mult', 'cost_diff', 'wait_time_x', 'wait_time_y' , 'wait_time_x', 'wait_time_y'
            group = group[['cost_mult', 'cost_diff', 'wait_time_x', 'wait_time_y', 'uniform_arrival_sweep_x', 'uniform_arrival_sweep_y', 'total_cloud_cost_x', 'total_cloud_cost_y']]
            
            display(group)
        
    for idx, (label, grp) in enumerate(groups):
        marker = next(markers)
#         if 'waiting_policy' in groupby_values[0]:
#             label = [legend_dict[label[0]]] + list(label[1:])
#             print(label)
        for ax_idx, ax in enumerate(axs):           
            grp.plot(x = x_axis, y = mod_y_axis[ax_idx],ylabel=label_dict[y_axis[ax_idx]], \
                     xlabel=label_dict[x_axis], marker=marker, ax = ax, label = label, legend=None)
    
    if return_df: 
        return diff_df 
    for ax in axs:
        set_plotting_setting(ax)
    #axs[1].set_xlim(right=36, left=20)
    lines, labels = ax.get_legend_handles_labels()
    fig.legend(lines, labels, ncol=len(labels), \
               bbox_to_anchor=(0, 0.92, 1, 0.2),loc='upper center')
    plt.tight_layout()
    plt.show()
    
'''
# TODO: Add spec to increase plot size
# TODO: pull the first distribution value
# TODO: retrieve sweep value based on the log value 
# TODO: Clean up ipynb to avoid overwriting jobs_df values
'''
def generate_system_util_plots(event_number=None, scale=4):
    events_dict, sweep_dict = log_jobs.retrieve_events_df(event_number=logs, avoid_congestion=False, only_dict=True)
    sweep_dict = OrderedDict(sweep_dict['varying_values'])
    
    # Make sure baseline filter is one row in dataframe
    # Make sure the baseline filter subtracts each corresponding arrival_rate
    for sweep_dim in sweep_dict: 
        for sweep_value in sweep_dict[sweep_dim]:
            print(sweep_dim + "_sweep")
            print(sweep_value)
            df = simulator_plotting_fn(jobs_df, \
                                       #x_axis='arrival_rate', \
                                       x_axis='uniform_arrival', \
                                       y_axis=['avg_jct', 'cluster_utilization', 'total_cloud_cost', 'cost_mult', 'cost_diff'], \
                                       baseline_filter= {'wait_time': 2}, \
                                       groupby_values=['wait_time'], \
                                       #'uniform_submission_sweep', 
                                       #'arrival_rate': 'Arrival Rate',
                                       #'uniform_arrival_sweep',
                                       normalize_x_axis=True, \
                                       intermediate_df={"baseline_df": False, "merged_df": True, "metrics_df": False, "groups_df": True}, \
                                       fig_ratio=(5*scale, 3.5*scale), \
                                       return_df=False)
            display(df)
            break 
        break
        
generate_system_util_plots(event_number=logs, scale=2)